In [4]:
import cPickle
import os.path

api_key = cPickle.load( file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'r' ) )

In [5]:
import cPickle
import os.path

cPickle.dump( api_key, file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'wb' ) )

In [6]:
import csv, pandas as pd
fb_table = pd.DataFrame.from_csv(os.path.expanduser('~/Downloads/top500.csv'))


In [7]:
import mediacloud, json
mc = mediacloud.api.MediaCloud(api_key)


In [9]:
all_media = []

last_media_id = 0

while True:
    media = mc.mediaList(last_media_id = last_media_id, rows = 1000)
    print len(all_media)
    if len( media ) == 0:
        break
        
    last_media_id = media[-1]['media_id']
    all_media += media
    
#the code below (commented out) only got 1000 media sources; 
#these 1000 media sources didn't contain many matches (~40) for the list of sites with facebook alignment data.
#my code gets about 260,000 media sources (yikes) and then manages to find matches for 419 of the 500 facebook news outlets.

# while True:
#     media = mc.mediaList( last_media_id=last_media_id, rows=1000)
#     #print last_media_id, len( media ), len( all_media )
#     print len(media)
#     if len(media) == 0:
#         break
        
#     last_media_id = media[-1]['media_id']
#     last_media_id
    

        
#     all_media.extend(media)
    
#     if len( all_media ) > 200:
#        break

len(all_media)
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


263217

In [ ]:
import urlnorm
from urlparse import urlparse
all_media_clean = {}

for i in all_media:
    url = i['url']
    media_id = i['media_id']
    try:
        #matching cleaned/parsed urls from mediacloud to mediacloud media IDs.
        all_media_clean[str(urlparse(url).netloc)] = i['media_id']
    except:
        #these urls are exceptionally broken. it is justifiable to remove them; 
        #they don't work in browsers and i believe there's maybe 3 out of 260k. 
        print url
        all_media.remove(i)
        

In [29]:
for s, r in fb_table.iterrows():
    pass
    #break
print s
dict(r)

weaselzippers.us


{'avg_align': 1.0295000000000001,
 'l1': 0.00020000000000000001,
 'l2': 0.0,
 'n': 0.00059999999999999995,
 'r1': 0.13,
 'r2': 0.86919999999999997}

In [35]:
matches = []
matched_by_url_parser = []
matched_url_parser_count = 0

for s, r in fb_table.iterrows():
    #all_media_clean is a hashtable (O(1) lookup) -- if the cleaned version of a url in the facebook dataset is there,
    #we will find it and match it to its mediacloud ID.
    
    url_path = str(urlparse(str(s)).path)
    
    if url_path in all_media_clean:
        
        match = dict( r )
        match['media_id'] = all_media_clean[ url_path]
        match['domain'] = s
        matches.append(match )
print len(matches)

    

420


In [36]:
matches

[{'avg_align': -0.99839999999999995,
  'domain': 'readersupportednews.org',
  'l1': 0.17549999999999999,
  'l2': 0.82150000000000001,
  'media_id': 33824,
  'n': 0.0028,
  'r1': 0.00020000000000000001,
  'r2': 0.0},
 {'avg_align': -0.97099999999999997,
  'domain': 'act.weareultraviolet.org',
  'l1': 0.84799999999999998,
  'l2': 0.152,
  'media_id': 113989,
  'n': 0.0,
  'r1': 0.0,
  'r2': 0.0},
 {'avg_align': -0.96730000000000005,
  'domain': 'www.commondreams.org',
  'l1': 0.094500000000000001,
  'l2': 0.89439999999999997,
  'media_id': 125311,
  'n': 0.010800000000000001,
  'r1': 0.0001,
  'r2': 0.00020000000000000001},
 {'avg_align': -0.94869999999999999,
  'domain': 'jezebel.com',
  'l1': 0.047300000000000002,
  'l2': 0.94979999999999998,
  'media_id': 88712,
  'n': 0.0025999999999999999,
  'r1': 0.0,
  'r2': 0.00029999999999999997},
 {'avg_align': -0.94630000000000003,
  'domain': 'www.thenation.com',
  'l1': 0.1011,
  'l2': 0.89559999999999995,
  'media_id': 202915,
  'n': 0.0030

In [43]:
df = pd.DataFrame( matches)
df.to_csv(os.path.expanduser('~/Desktop/fb_align.csv'), 
          index=False, columns = ['media_id', 'domain', 'avg_align', 'l1','l2', 'n', 'r1', 'r2' ])

In [16]:
fb_table

,avg_align,l2,l1,n,r1,r2
domain,,,,,,
readersupportednews.org,-0.9984,0.8215,0.1755,0.0028,0.0002,0.0000
act.weareultraviolet.org,-0.9710,0.1520,0.8480,0.0000,0.0000,0.0000
www.commondreams.org,-0.9673,0.8944,0.0945,0.0108,0.0001,0.0002
www.thenewcivilrightsmovement.com,-0.9643,0.9869,0.0072,0.0059,0.0000,0.0000
jezebel.com,-0.9487,0.9498,0.0473,0.0026,0.0000,0.0003
ringoffireradio.com,-0.9472,0.9861,0.0134,0.0005,0.0000,0.0000
www.thenation.com,-0.9463,0.8956,0.1011,0.0030,0.0001,0.0002
www.rightwingwatch.org,-0.9407,0.8874,0.1046,0.0060,0.0020,0.0000
www.liberalamerica.org,-0.9387,0.9855,0.0145,0.0000,0.0000,0.0000
